In [1]:
import rospy
import jupyros as jr
import jupyros_action_client_patched as jr_patched
import ipywidgets as widgets
from IPython.display import display
import matplotlib as mpl
import matplotlib.pyplot as plt

import tf
from turtlesim.msg import Pose
from tf.transformations import quaternion_matrix
import numpy as np
from matplotlib.animation import FuncAnimation

In [2]:
from assignment_2_2023.msg import PlanningGoal, PlanningAction
from monitoring_package.msg import poseVelocity

In [3]:
rospy.init_node('jupyter_node')
# r = rospy.Rate(10) # 10hz

In [4]:
cancelled_goals_html = widgets.HTML(
    value = ''
)
sent_goal_html = widgets.HTML(value = '')


In [5]:
goals_achieved = 0
goals_cancelled = 0

x_goal, y_goal = (0, 0)
new_goal_flag = False

cancelled_goals = []
achieved_goals = []
def action_feedback_callback(e):
    global goals_cancelled, goals_achieved, cancelled_goals, cancelled_goals_html, achieved_goals, x_goal, y_goal
#     if (e.stat.strip() == 'Target cancelled!'):
#         rospy.loginfo('feedback received:')
#         goals_cancelled+=1
#         cancelled_goals.append((x_goal, y_goal)) 
#         cancelled_goals_html.value="<br>".join(f"Pose {i+1}: ({cancelled_goals[i][0]}, {cancelled_goals[i][1]})" for i in range(0, len(cancelled_goals)))
        
        
    if (e.stat.strip() == 'Target reached!'):
        goals_achieved+=1
        achieved_goals.append((x_goal, y_goal))
def cancelled_callback():
    global goals_cancelled, x_goal, y_goal
    rospy.loginfo('feedback received:')
    goals_cancelled+=1
    cancelled_goals.append((x_goal, y_goal)) 
    cancelled_goals_html.value="<br>".join(f"Pose {i+1}: ({cancelled_goals[i][0]}, {cancelled_goals[i][1]})" for i in range(0, len(cancelled_goals)))
        
        
def sent_goal_callback(m):
    global x_goal, y_goal, new_goal_flag
    x_goal = m.target_pose.pose.position.x
    y_goal = m.target_pose.pose.position.y
    new_goal_flag = True
    
    


In [6]:
jr_patched.action_client('reaching_goal', PlanningAction, PlanningGoal, callbacks={'feedback_cb': action_feedback_callback, 'last_sent_goal_cb': sent_goal_callback, 'cancel_cb': cancelled_callback})

[INFO] [1717191501.104272, 3653.687000]: [REACHING_GOAL] Waiting for action server.
[INFO] [1717191501.157667, 3653.741000]: [REACHING_GOAL] Connection to server successful.
last_sent_goal_cb[ <function sent_goal_callback at 0x7f8b7edb5b80>]
[INFO] [1717191501.159775, 3653.741000]: last_sent_goal_cb[ <function sent_goal_callback at 0x7f8b7edb5b80>]


In [7]:


MAX_SIZE = 10000
class Visualiser:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'ro')
        self.ln2, = plt.plot([], [], 'bs')
        self.x_data, self.y_data = [] , []
        self.x_goal, self.y_goal = [], []

    def plot_init(self):
        self.ax.set_xlim(-10, 10)
        self.ax.set_ylim(-10, 10)
        return self.ln, self.ln2

    def poseCallback(self, pV):
        global x_arr, y_arr

        if (len(self.x_data) > 0 and len(self.y_data) > 0):
            if (((self.x_data[-1]-pV.x)**2 + (self.y_data[-1]-pV.y)**2)**0.5 < 0.1):
                self.x_data.append(pV.x)
                self.y_data.append(pV.y)

            if (len(self.x_data)>MAX_SIZE):
                self.x_data = self.x_data[len(self.x_data)//2:]
                self.y_data = self.y_data[len(self.y_data)//2:]


        else:
            self.x_data.append(pV.x)
            self.y_data.append(pV.y)

        global x_goal, y_goal, new_goal_flag, sent_goal_html
        if (new_goal_flag):
            self.x_goal.append(x_goal)
            self.y_goal.append(y_goal)
            sent_goal_html.value="<br>".join(f"Pose {i+1}: ({self.x_goal[i]}, {self.y_goal[i]})" for i in range(len(self.x_goal)))

            new_goal_flag = False
        
        self.update_current_location()
        
    def update_current_location(self):
        global x_current_widget, y_current_widget
        x_current_widget.value = self.x_data[-1]
        y_current_widget.value = self.y_data[-1]
        

    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        
        xg, yg = [], []
        try:
            xg = self.x_goal[-1]
            yg = self.y_goal[-1]
            
        except:
            pass
        
        self.ln2.set_data(xg, yg)
        return self.ln, self.ln2



In [8]:
label = widgets.Label(value = 'current pose' )
display(label)
x_current_widget = widgets.FloatText(value=0, description='x:', disabled=True)
display(x_current_widget)
y_current_widget = widgets.FloatText(value=0, description='y:', disabled=True)
display(y_current_widget)

Label(value='current pose')

FloatText(value=0.0, description='x:', disabled=True)

FloatText(value=0.0, description='y:', disabled=True)

In [9]:
# Create the animation
%matplotlib widget
# Create a new figure and add a subplot

vis = Visualiser()
rospy.Subscriber("/current_pose_velocity_publisher", poseVelocity, callback=vis.poseCallback)
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)
plt.title('Position of robot and latest goal')
plt.legend(['Position of robot', 'goal'])
plt.show(block=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
set_goal_label = widgets.Label(value = 'Set goals:')
display(set_goal_label)

display(sent_goal_html)

set_goal_label = widgets.Label(value = 'Cancelled goals:')
display(set_goal_label)

display(cancelled_goals_html)

Label(value='Set goals:')

HTML(value='')

Label(value='Cancelled goals:')

HTML(value='')

In [15]:
fig, ax = plt.subplots()

bp = None
def cb(frame):
    global vis, cancelled_goals, bp, fig, ax, achieved_goals
    xlabels = ['Sent', 'Cancelled', 'Achieved']
    count = [len(vis.x_goal), len(cancelled_goals), len(achieved_goals)] 
    ax.cla()
    # Redraw box plot
    bp = ax.bar(xlabels, count, color='blue', width=0.6)
    plt.show()
    
    return bp,
    
    
    

ani2 = FuncAnimation(fig, cb)
# plt.show(block=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

AttributeError: 'Figure' object has no attribute 'title'